In [13]:
import pandas as pd
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

In [14]:
df = pd.read_csv('Results_21Mar2022.csv')
df['diet_group'] = df['diet_group'].replace({
    'meat50': 'meat<50',
    'meat': 'meat50-99',
    'meat100': 'meat100+'
})
df.head()

,mc_run_id,grouping,mean_ghgs,mean_land,mean_watscar,mean_eut,mean_ghgs_ch4,mean_ghgs_n2o,mean_bio,mean_watuse,...,sd_eut,sd_ghgs_ch4,sd_ghgs_n2o,sd_bio,sd_watuse,sd_acid,n_participants,sex,diet_group,age_group
0,1,fish_female_20-29,5.421821,8.612772,14165.86240,23.945372,1.164643,0.343663,476.270413,679.923479,...,8.167013,0.632547,0.146827,283.883428,249.472563,8.231176,1329,female,fish,20-29
1,1,fish_female_30-39,5.436673,8.680248,15222.77063,24.305262,1.154118,0.326868,467.923201,709.343785,...,8.467858,0.601941,0.130298,270.247108,277.927269,7.947014,2024,female,fish,30-39
2,1,fish_female_40-49,5.289333,8.467723,15818.80068,24.191105,1.108404,0.310324,438.659832,707.925368,...,8.107006,0.572941,0.119872,252.415063,275.760515,8.103589,1689,female,fish,40-49
3,1,fish_female_50-59,5.169020,8.219199,16962.06009,24.748154,1.032501,0.298130,395.787622,720.825105,...,9.514028,0.509689,0.104731,209.566722,310.422445,8.712428,946,female,fish,50-59
4,1,fish_female_60-69,5.380268,8.570718,18212.66232,25.559430,1.098828,0.310735,421.176867,749.392826,...,9.258068,0.609398,0.113401,264.599257,315.922570,8.616601,492,female,fish,60-69


In [15]:
environmental_metrics = ['mean_ghgs', 'mean_land', 'mean_watscar', 'mean_eut', 'mean_bio', 'mean_acid']

In [16]:
# Aggregate by dietary type and take the average
df_grouped = df.groupby('diet_group')[environmental_metrics].mean().reset_index()

In [17]:
# Standardization
scaler = MinMaxScaler()
df_grouped_scaled = df_grouped.copy()
df_grouped_scaled[environmental_metrics] = scaler.fit_transform(df_grouped[environmental_metrics])

In [18]:
# Calculate the environmental burden score, the higher the score, the less environmentally friendly the diet is
df_grouped_scaled['total_score'] = df_grouped_scaled[environmental_metrics].sum(axis=1)

In [19]:
best_diet = df_grouped_scaled.loc[df_grouped_scaled['total_score'].idxmin()]
worst_diet = df_grouped_scaled.loc[df_grouped_scaled['total_score'].idxmax()]

In [20]:
categories = environmental_metrics + [environmental_metrics[0]]
best_values = best_diet[environmental_metrics].tolist() + [best_diet[environmental_metrics[0]]]
worst_values = worst_diet[environmental_metrics].tolist() + [worst_diet[environmental_metrics[0]]]

In [30]:
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=best_values,
    theta=categories,
    name=f"The most environmentally friendly diet: {best_diet['diet_group']}",
    line=dict(color='#DD7650'),
    fill='toself'
))

fig.add_trace(go.Scatterpolar(
    r=worst_values,
    theta=categories,
    name=f"The least environmentally friendly diet: {worst_diet['diet_group']}",
    line=dict(color='#7D927D'),
    fill='toself'
))

# 图像布局
fig.update_layout(
    polar=dict(
        radialaxis=dict(visible=True, range=[0, 1])
    ),
    title="Comparison: Most vs Least Eco-Friendly Diets",
    showlegend=True
)

fig.show()